In [20]:
!nvidia-smi

Sun Aug 13 21:26:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    28W /  70W |   3747MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
! pip install -q langchain transformers sentence_transformers llama-index

In [22]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor, ServiceContext
from llama_index import GPTVectorStoreIndex
import torch
from langchain.llms.base import LLM
from transformers import pipeline


In [23]:


class customLLM(LLM):
    model_name = "google/flan-t5-large"
    pipeline = pipeline("text2text-generation", model=model_name, device=0, model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]

    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custom"


llm_predictor = LLMPredictor(llm=customLLM())



In [24]:
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)

In [25]:
import logging

logging.getLogger().setLevel(logging.CRITICAL)

In [26]:
# temp fix for running shell commands on Google Colab

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [27]:
!pip install gradio -q

In [28]:
import gradio as gr

In [29]:
index = None

In [30]:
from llama_index import Document
def build_the_bot(input_text):
  text_list = [input_text]
  documents = []
  for t in text_list:
    doc = Document()
    #doc.content = t
    documents.append(doc)
  #documents = [Document(t) for t in text_list]
  global index
  service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model)
  index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)
  return('Index saved successfully!!!')

In [31]:



  def chat(chat_history, user_input):
    query_engine = index.as_query_engine()
    bot_response = query_engine.query(user_input)

    if bot_response and bot_response.response:
        response = ""

        if isinstance(bot_response.response, list):
            for item in bot_response.response:
                if isinstance(item, str):
                    response += item
        elif isinstance(bot_response.response, str):
            response = bot_response.response

        return chat_history + [(user_input, response)]
    else:
        return chat_history
    #response = bot_response.response)
    #response = ""

    #for letter in bot_response.response:
     # response += letter + ""

      #return chat_history + [(user_input, response)]





In [ ]:

with gr.Blocks() as demo:
    gr.Markdown('# Q&A Bot with Hugging Face Models')
    with gr.Tab("Input Text Document"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Build the Bot!!!")
        text_button.click(build_the_bot, text_input, text_output)
    with gr.Tab("Knowledge Bot"):
#          inputbox = gr.Textbox("Input your text to build a Q&A Bot here.....")
          chatbot = gr.Chatbot()
          message = gr.Textbox ("What is this document about?")
          message.submit(chat, [chatbot, message], chatbot)

demo.queue().launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1fe19ebb16211096a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
